### Tensorflow tutorial
https://www.tensorflow.org/get_started/get_started

In [30]:
import numpy as np
import tensorflow as tf

In [2]:
# константы
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)   # implicitly float32
print node1, node2

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [3]:
# запуск графа на исполнение требует создания сессии
sess = tf.Session()
print sess.run([node1, node2])

[3.0, 4.0]


In [12]:
# операции над тензорами тоже являются нодами
node3 = tf.add(node1, node2)
print node3
print sess.run(node3)

Tensor("Add_2:0", shape=(), dtype=float32)
7.0


In [16]:
# входные данные задаются с помощью узлов типа placeholder
a = tf.placeholder(dtype=tf.float32)
b = tf.placeholder(dtype=tf.float32)
adder_node = a + b

print sess.run(adder_node, {a: 3, b: 5})
print sess.run(adder_node, {a: [1, 2], b: [3, 5]})

8.0
[ 4.  7.]


In [22]:
# ноды типа Variable, значения которых могут меняться (мутабельные)
w = tf.Variable(initial_value=[0.3], dtype=tf.float32)
b = tf.Variable(initial_value=[-0.3], dtype=tf.float32)
x = tf.placeholder(dtype=tf.float32)
linear_model = w * x + b

# в отличие от констант, переменные в тензорфлоу требуют явной инициализации
init = tf.global_variables_initializer()
sess.run(init)

# граф можно выполнить для нескольких входных значений плейсхолдера
print sess.run(linear_model, {x: [0.0, 1.0, 2.0]})

[-0.30000001  0.          0.30000001]


In [23]:
# целевая функция для линейной модели: квадратичная ошибка
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)

print sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

23.66


In [24]:
# вручную изменить значение переменной ноды можно с помощью операции assign 
fix_w = tf.assign(w, [-1.0])
fix_b = tf.assign(b, [1.0])
sess.run([fix_w, fix_b])

print sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

0.0


In [29]:
# использование автоматических градиентов и оптимизатора
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

# реинициализируем переменные
sess.run(init)

# обучаем 
for i in xrange(1000):
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
    
# выводим результат
print sess.run([w, b, loss], {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32), 5.6999738e-11]


In [34]:
# более высокоуровневое API: tf.estimator

# создаем список фичей (в случае одномерной линейной регресии используем одну фичу)
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# создаем экземпляр модели
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# использование numpy
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print "train metrics: %r"% train_metrics
print "eval metrics: %r"% eval_metrics


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x119c36590>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/c3/tly0db3j0j1_t1vgkhvtw0jnhg_gr1/T/tmpB26VWL', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/c3/tly0db3j0j1_t1vgkhvtw0jnhg_gr1/T/tmpB26VWL/model.ckpt.
INFO:tensorflow:loss = 18.0, step = 1
INFO:tensorflow:global_step/sec: 966.314
INFO:tensorflow:loss = 0.122829, step = 101 (0.107 sec)
INFO:tensorflow:global_step/sec: 1136.27
INFO:tensorflow:loss = 0.0414071, step = 201 

TypeError: Fetch argument <tensorflow.python.estimator.canned.linear.LinearRegressor object at 0x119c83390> has invalid type <class 'tensorflow.python.estimator.canned.linear.LinearRegressor'>, must be a string or Tensor. (Can not convert a LinearRegressor into a Tensor or Operation.)

In [37]:
def model_fn(features, labels, mode):
    # Build a linear model and predict values
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W*features['x'] + b
    # Loss sub-graph
    loss = tf.reduce_sum(tf.square(y - labels))
    # Training sub-graph
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
    # EstimatorSpec connects subgraphs we built to the
    # appropriate functionality.
    return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11ac9a710>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/c3/tly0db3j0j1_t1vgkhvtw0jnhg_gr1/T/tmpJzVthX', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/c3/tly0db3j0j1_t1vgkhvtw0jnhg_gr1/T/tmpJzVthX/model.ckpt.
INFO:tensorflow:loss = 53.6989671039, step = 1
INFO:tensorflow:global_step/sec: 995.312
INFO:tensorflow:loss = 0.661202087612, step = 101 (0.103 sec)
INFO:tensorflow:global_step/sec: 1153.19
INFO:tensorflow:loss = 0.06333